In [ ]:
!pip install pytorch_pretrained_bert pytorch-nlp

In [59]:
from pytorch_pretrained_bert import BertTokenizer
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv

In [61]:
batch_size = 10
sequence_length = 510
max_char_length = 5000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [62]:
class Preprocessor:

    def run(self, sentence):
        sentence = self.remove_before_word(sentence, '@')
        sentence = sentence.replace("\\n", ' ')
        sentence = sentence.replace("\\", '')
        return sentence

    def remove_before_word(self, sentence, word):
        try:
            subject_index = sentence.index(word)
            sentence = sentence[subject_index + len(word):]
        except:
            print(f"HEY! No word '{word}' in {sentence}")
        return sentence

preprocessor = Preprocessor()

In [64]:
test_dataset = text_dataset_from_directory('../data/test', label_mode='categorical', batch_size=batch_size, max_length=max_char_length, shuffle=False)

Found 7761 files belonging to 20 classes.


In [70]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

with open('test_data_bert.csv', 'w') as data_file:
    writer = csv.writer(data_file)
    i = 0
    for batch in test_dataset:
        texts, labels = batch
        clean_texts = [preprocessor.run(text.decode("utf-8")) for text in texts.numpy()]
        test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:sequence_length] + ['[SEP]'], clean_texts))
        sequences = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
        sequences = pad_sequences(sequences, maxlen=512, truncating=trunc_type, padding=padding_type, dtype="int")
        writer.writerows(list(sequence) + list(label.numpy().astype(int)) for sequence, label in zip(sequences, labels))
        i += 1
        if i > 1:
            break

! cat test_data_bert.csv

101,21824,7974,2015,3501,1012,17324,1012,2012,2102,1012,4012,1006,4670,1012,10556,12314,8838,1007,3395,1024,2128,1024,2055,1996,6331,19461,6998,3029,1024,2012,1004,1056,4353,1024,6583,3210,1024,2324,1999,3720,1026,11005,2696,1012,16710,1012,6421,25746,20958,22394,2581,1030,14784,1012,1059,16526,1012,3968,2226,1028,1010,11005,2696,1030,14784,1012,1059,16526,1012,3968,2226,1006,19971,1054,25706,1007,7009,1024,1028,1028,1028,1001,2260,1007,1996,1016,24188,12322,18163,2024,2006,1996,15745,1997,1996,16077,1012,2043,2643,2056,2191,2053,1028,6542,2078,3746,1010,2002,2001,6523,2075,2000,24438,1010,2029,2020,2580,2000,2022,22876,1012,1028,1996,15745,1997,1996,16077,2347,1005,1056,23277,7716,5605,5669,1998,2069,1996,2152,5011,2071,1028,4607,1996,4151,1997,7570,11983,2073,2009,2001,2921,2320,1037,2095,1010,2006,1996,2154,1997,1028,2012,5643,3672,1012,1045,2572,2025,5220,2007,1010,2030,3716,3085,2055,1996,2434,2653,1010,2021,1045,2903,2045,2003,1037,2773,2005,1000,10282,1000,1998,2008,1996,11403,2